# 감성분석 회귀분석 예측

<p> 0 : 기쁨 행복</p>
<p> 1 : 분노 불안 당황 놀람</p>
<p> 2: 슬픔 상처 </p>

In [40]:
from sklearn.model_selection import train_test_split
import pandas as pd
from afinn import Afinn
from nltk.corpus import stopwords
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

dataset = pd.read_csv("sentiment_analysis.csv",encoding = 'utf-8')
dataset.head()

,number,value,age,sex,keyword,disease,sentiment_big,sentiment_small,person1,system1,person2,system2,person3,system3
0,24070,1,청년,여성,"진로, 취업, 직장",해당없음,기쁨,신이 난,지금 난 기분이 너무 좋아.,굉장히 즐거우신 것 같은데요? 좋은 일이 있나요?,오늘부터 연휴가 시작됐거든. 내일 회사 안 간다.,그거 정말 기분 좋을 만하네요. 무엇을 할 예정인가요?,그냥 집에서 뒹굴뒹굴하고 넷플릭스 보고 놀 거야.,집에서 노는 것이 최고죠. 생각만 해도 행복하겠어요.
1,27879,1,청년,여성,"연애, 결혼, 출산",해당없음,기쁨,신이 난,나도 조카가 생겨! 너무 기뻐.,정말 좋은 소식이네요. 축하할 특별한 계획이 있을까요?,응. 언니에게 꽃다발을 선물할 거야. 예쁜 딸이면 좋을 것 같아서.,언니에게 꽃을 선물하려고 하시는군요.,NaN,NaN
2,51471,1,중년,여성,"직장, 업무 스트레스",해당없음,불안,불안,거래처와의 다음 계약이 무산될까봐 불안해.,많이 걱정되시겠어요. 계약이 무산될 것 같다고 생각하는 이유가 있으신가요?,딱히 이유는 없어. 그냥 나 혼자 불안한 상태야.,계약을 성공적으로 마무리하실 수 있기를 기원해요.,NaN,NaN
3,7140,1,노년,남성,대인관계,만성질환 유,슬픔,염세적인,당뇨로 고생 중인데 아내가 이혼하자더군. 아내가 날 버리면 병은 더 악화될 텐데 세...,몸이 아픈데 아내가 이혼하자고 하니 혼자 있는 것 같은 기분을 느끼셨군요.,젊은 시절에 가정에 소홀했지. 그래도 그렇지 이혼을 하자니 너무 허망해.,지금의 기분에서 벗어나기 위해 무엇을 하면 좋을까요?,아내에게 졸혼은 어떤지 대화를 나누어 봐야겠어. 졸혼을 한다면 가끔씩이라도 도움을 ...,아내와의 대화로 기분이 나아졌으면 좋겠어요.
4,39663,1,노년,남성,재정,만성질환 유,슬픔,마비된,노년에 재정적으로 이루어 놓은게 없어서 걱정이야.,나이들어 재정적으로 어려워서 걱정되시는군요.,정말이지 막막해.,막막한 현실이지만 분명 해결책을 찾을 수 있을거예요.,NaN,NaN


In [41]:
sentimential_list = {'행복' : 0, '기쁨': 0, '분노' : 1, '불안' : 1, '당황' : 1, '슬픔' : 2, '상처' : 2}
dataset['label'] = dataset['sentiment_big'].map(sentimential_list)
dataset.head()

,number,value,age,sex,keyword,disease,sentiment_big,sentiment_small,person1,system1,person2,system2,person3,system3,label
0,24070,1,청년,여성,"진로, 취업, 직장",해당없음,기쁨,신이 난,지금 난 기분이 너무 좋아.,굉장히 즐거우신 것 같은데요? 좋은 일이 있나요?,오늘부터 연휴가 시작됐거든. 내일 회사 안 간다.,그거 정말 기분 좋을 만하네요. 무엇을 할 예정인가요?,그냥 집에서 뒹굴뒹굴하고 넷플릭스 보고 놀 거야.,집에서 노는 것이 최고죠. 생각만 해도 행복하겠어요.,0
1,27879,1,청년,여성,"연애, 결혼, 출산",해당없음,기쁨,신이 난,나도 조카가 생겨! 너무 기뻐.,정말 좋은 소식이네요. 축하할 특별한 계획이 있을까요?,응. 언니에게 꽃다발을 선물할 거야. 예쁜 딸이면 좋을 것 같아서.,언니에게 꽃을 선물하려고 하시는군요.,NaN,NaN,0
2,51471,1,중년,여성,"직장, 업무 스트레스",해당없음,불안,불안,거래처와의 다음 계약이 무산될까봐 불안해.,많이 걱정되시겠어요. 계약이 무산될 것 같다고 생각하는 이유가 있으신가요?,딱히 이유는 없어. 그냥 나 혼자 불안한 상태야.,계약을 성공적으로 마무리하실 수 있기를 기원해요.,NaN,NaN,1
3,7140,1,노년,남성,대인관계,만성질환 유,슬픔,염세적인,당뇨로 고생 중인데 아내가 이혼하자더군. 아내가 날 버리면 병은 더 악화될 텐데 세...,몸이 아픈데 아내가 이혼하자고 하니 혼자 있는 것 같은 기분을 느끼셨군요.,젊은 시절에 가정에 소홀했지. 그래도 그렇지 이혼을 하자니 너무 허망해.,지금의 기분에서 벗어나기 위해 무엇을 하면 좋을까요?,아내에게 졸혼은 어떤지 대화를 나누어 봐야겠어. 졸혼을 한다면 가끔씩이라도 도움을 ...,아내와의 대화로 기분이 나아졌으면 좋겠어요.,2
4,39663,1,노년,남성,재정,만성질환 유,슬픔,마비된,노년에 재정적으로 이루어 놓은게 없어서 걱정이야.,나이들어 재정적으로 어려워서 걱정되시는군요.,정말이지 막막해.,막막한 현실이지만 분명 해결책을 찾을 수 있을거예요.,NaN,NaN,2


In [49]:
import re
dataset['text'] = dataset['person1'].map(str) + dataset['system1'].map(str) + dataset['person2'].map(str)+ dataset['system2'].map(str) + dataset['person3'].map(str) + dataset['system3'].map(str)
new_data = {"text" : "ㅎㅎ","label": 0}
dataset = dataset.append(new_data, ignore_index = True)

x_data = dataset['text']
y_data = dataset['label'].values

x_data = x_data.apply(lambda x : re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣ\\s]', " ", x)).values
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, stratify = y_data, test_size = 0.2, random_state = 42)


from konlpy.tag import Okt

okt = Okt()
def tw_tokenizer(text):
    tokens_ko = okt.morphs(text)
    return tokens_ko

In [43]:
# label 감정별 데이터 분포

dataset['label'].value_counts()

1    36194
2    26213
0    12449
Name: label, dtype: int64

In [7]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import f1_score, roc_auc_score
import time

In [8]:
from lightgbm import LGBMClassifier

def light_gbm_train(x_train_okt, x_test_okt, y_train, y_test):
    print("LGBM Classifier\n")
    start_time = time.time()
    lgbm_clf = LGBMClassifier(n_estimators = 400, n_jobs=-1, reg_alpha = 0.9, reg_lambda = 0.2)
    lgbm_clf.fit(x_train_okt, y_train)
    preds = lgbm_clf.predict(x_test_okt)
    confusion = confusion_matrix(y_test, preds)
    accuracy = accuracy_score(y_test, preds)
    precision = precision_score(y_test, preds, average='weighted')
    recall = recall_score(y_test, preds, average='weighted')
    f1 = f1_score(y_test, preds, average='weighted')
    
    print(confusion)
    print('정확도: {0:.4f}\n정밀도: {1:.4f}\n재현율: {2:.4f}\nf1: {3:.4f}'.format(accuracy,precision,recall,f1))
    print(f'fit time : {time.time() - start_time}')

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(ngram_range = (1, 2), min_df = 3, max_df = 0.9, tokenizer = okt.morphs, token_pattern = None)
tfidf.fit(x_train)
x_train_okt = tfidf.transform(x_train)
x_test_okt = tfidf.transform(x_test)

light_gbm_train(x_train_okt, x_test_okt, y_train, y_test)

LGBM Classifier

[[2152  258   80]
 [ 114 6239  886]
 [  82 1448 3713]]
정확도: 0.8084
정밀도: 0.8100
재현율: 0.8084
f1: 0.8074
fit time : 254.94550371170044


<p> 0 : 기쁨 행복</p>
<p> 1 : 분노 불안 당황 놀람</p>
<p> 2: 슬픔 상처 </p>

In [10]:
lgbm_clf = LGBMClassifier(n_estimators = 400, n_jobs=-1, reg_alpha = 0.9, reg_lambda = 0.2)
lgbm_clf.fit(x_train_okt, y_train)
light_gbm_train(x_train_okt, x_test_okt, y_train, y_test)

LGBM Classifier

[[2152  258   80]
 [ 114 6239  886]
 [  82 1448 3713]]
정확도: 0.8084
정밀도: 0.8100
재현율: 0.8084
f1: 0.8074
fit time : 254.15628862380981


In [11]:
x_test = "ㅎㅎ"

In [12]:
lgbm_clf.predict(tfidf.transform([x_test]))

array([1], dtype=int64)

In [13]:
from selenium import webdriver
from selenium.webdriver.common.by import By 
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC 
from bs4 import BeautifulSoup
import time 
import re
import json 
import pandas as pd
from datetime import datetime

In [14]:
## id 와 password 입력

user_id=
user_passwd=

In [15]:
login_option="instagram" # facebook or instagram 
driver_path="chromedriver.exe" 
instagram_id_name="username" 
instagram_pw_name="password"
instagram_login_btn=".sqdOP.L3NKy.y3zKF "
instagram_information_save_name = ".sqdOP.yWX7d.y3zKF"
instagram_alarm_name = ".aOOlW.HoLwm"
is_information_save = False
is_alarm_setting = False

driver = webdriver.Chrome("./chromedriver")

In [16]:
print(f"login start - option {login_option}")
login_url = "https://www.instagram.com/accounts/login/"
driver.get(login_url)
print(login_option)

if login_option == "instagram": 
    time.sleep(2)
    try: 
        instagram_id_form = driver.find_element_by_name(instagram_id_name) 
        instagram_id_form.send_keys(user_id) 
        
        instagram_pw_form = driver.find_element_by_name(instagram_pw_name) 
        instagram_pw_form.send_keys(user_passwd) 
        
        login_ok_button = driver.find_element_by_css_selector(instagram_login_btn) 
        login_ok_button.click() 
        is_login_success = True
        
        time.sleep(3)
        if is_information_save == False and is_alarm_setting == False:
            information_save_btn = driver.find_element_by_css_selector(instagram_information_save_name)
            information_save_btn.click()
            is_information_save = True
            
            alarm_setting_btn = driver.find_element_by_css_selector(instagram_alarm_name)
            alarm_setting_btn.click()
            is_alarm_setting = True
            
    except: 
        print("instagram login fail") 
        is_login_success = False

login start - option instagram
instagram


In [17]:
## 위 내용까지 로그인

my_account_xpath = "/html/body/div[1]/section/nav/div[2]/div/div/div[3]/div/div[6]/span"
my_account_button = driver.find_element_by_xpath(my_account_xpath)
my_account_button.click()

my_profile_full_xpath = "/html/body/div[1]/section/nav/div[2]/div/div/div[3]/div/div[6]/div[2]/div[2]/div[2]/a[1]/div"
my_profile_button = driver.find_element_by_xpath(my_profile_full_xpath)
my_profile_button.click()

time.sleep(1.5)
# 게시물 개수
post_count = driver.find_element_by_xpath("/html/body/div[1]/section/main/div/header/section/ul/li[1]/div/span").text
post_count = int(post_count)

print("게시물 수 : ",post_count)

insta_df = pd.DataFrame()
new_data = {"date" : "날짜", "place":"장소","content":"본문내용","hashtag":"해시태그","comment":"댓글내용"}
insta_df = insta_df.append(new_data,ignore_index=True)
insta_df.head()

게시물 수 :  12


,date,place,content,hashtag,comment
0,날짜,장소,본문내용,해시태그,댓글내용


### 현재 모든 댓글보기 클릭 가능 (while문 이용)
<li> 1. 모든 답글이 보여진 상태에서 html parser 이용
<li> 2. css 또는 xpath 이용

In [20]:
# 첫 게시물
first_post_css = "a > div.eLAPa > div._9AhH0"
first_post_button = driver.find_element_by_css_selector(first_post_css)
first_post_button.click()

for i in range(post_count) :
    # 날짜 데이터 크롤링
    time.sleep(2)
    date = driver.find_element_by_css_selector('time._1o9PC')
    date_info = pd.to_datetime(date.get_attribute('datetime')).normalize()
    date = str(date_info.year) + "년 " + str(date_info.month) + "월 " + str(date_info.day) +"일"

    # 장소 크롤링
    time.sleep(0.5)
    try : 
        place = driver.find_element_by_css_selector("body > div.RnEpo._Yhr4 > div.pbNvD.QZZGH.bW6vo > div > article > div > div.HP0qD > div > div > div.UE9AK > div > header > div.o-MQd.z8cbW > div.M30cS > div > div.JF9hh > div > a").text
    except : 
        place = None

    try : 
        post_content = driver.find_element_by_xpath("/html/body/div[6]/div[3]/div/article/div/div[2]/div/div/div[2]/div[1]/ul/div/li/div/div/div[2]/div[1]/span").text    
    except :
        post_content = None
    post_content_list =[]
    post_content_list.append(post_content)
   
    # 본문 속 해시태그 정제
    if post_content != None :
        post_content_tag= []
        post_content = re.findall("#[A-Za-z0-9가-힣]+", post_content)
        content = ''.join(post_content).replace("#"," ") # "# 제거"
        content_split = content.split()
        
        for tag in content_split :
            post_content_tag.append(tag)
    
    # 댓글 크롤링
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    comment_count = 0

    # 댓글 있는지 확인
    try  :
        ul_list = soup.find_all('ul',  'Mr508')
        comment_count = len(ul_list)
    except :
        # 댓글 없으면
        comment_count = 0
    else : 
        # 댓글 개수 적용하여 대댓글 열기버튼 xpath 추출
        for i in range(1,comment_count+1) :
            comment_list = []
            button_xpath = "/html/body/div[6]/div[3]/div/article/div/div[2]/div/div/div[2]/div[1]/ul/ul[" + str(i) + "]/li/ul/li/div/button"
            try : 
                # 대댓글 열기버튼 있으면 클릭
                driver.find_element_by_xpath(button_xpath).click()
                time.sleep(1)
                button_check = driver.find_element_by_class_name("EizgU").text
                while "답글 숨기기" not in button_check : 
                    driver.find_element_by_xpath(button_xpath).click()
                    button_check = driver.find_element_by_class_name("EizgU").text
                    
            except :
                button_xpath = None
            
        # 대댓글이 모두 열려있는 상태에서 html parser        
        # 추가 댓글이 모두 열려있는 상태에서 html parser        
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        ul_list = soup.find_all("ul","Mr508")
        for d in ul_list :
            user_id = d.find("h3","_6lAjh").find("a").get_text()

            comment = d.find("div","C4VMK").find("div","MOdxS").get_text()

            comment_list.append(user_id+":"+comment)
            try :
                re_ul_list = d.find_all("ul","TCSYW")
                for dd in re_ul_list :
                    #print("[ 답글시작 ]")
                    try : 
                        re_div_list = dd.find_all("div","ZyFrc")
                        for ddd in re_div_list :
                            #print("[답글의 답글 시작]")
                            comment_user_id = ddd.find("h3","_6lAjh").find("a").get_text()
                            
                            comment_comment = ddd.find("div","C4VMK").find("div","MOdxS").get_text()

                            comment_list.append(comment_user_id+":"+comment_comment)
                    except : 
                        re_div_list =""
            except : 
                re_ul_list = ""

    new_data = {"date" : date, "place":place, "content" : post_content_list,"hashtag" : post_content_tag,"comment" : comment_list}
    insta_df = insta_df.append(new_data,ignore_index=True)
    # 다음 버튼
    try :
        next_post_button = driver.find_element_by_css_selector("div.RnEpo._Yhr4 > div.Z2Inc._7c9RR > div > div.l8mY4.feth3 > button")
        next_post_button.click()
        time.sleep(0.5)
    except :
        # 닫기 버튼    
        exit_button = driver.find_element_by_css_selector("div.RnEpo._Yhr4 > div.NOTWr > button > div > svg")
        exit_button.click()
        break

In [81]:
insta_df

,date,place,content,hashtag,comment,label
0,날짜,장소,본문내용,해시태그,댓글내용,1
1,2022년 2월 24일,어마무시 티라미수,[2022❤️],[],"[zzae_v:다 좋은데 신발은…?, go_ne_jin:@zzae_v 벗어 던져버렸...",1
2,2021년 8월 29일,낙산사,[개강 전 finale\n.\n.\n.\n.\n.\n.\n.\n.\n.\n.\n혜진...,[],"[suuout:👶, go_ne_jin:@sweeni_54 뻬이뻬 K대면에서 만나…,...",1
3,2021년 7월 11일,하이서울유스호스텔 Hi Seoul Youth Hostel,[나도 기본캠📸플래쉬 감성 ?\n.\n.\n.\n.\n.\n.\n.\n.\n포둉포둉...,[],"[perarduaadastra__:어디가 포동포동이야 전혀 😮, go_ne_jin:...",1
4,2021년 7월 11일,하이서울유스호스텔 Hi Seoul Youth Hostel,[글램핑\n.\n.\n.\n.\n.\n.\n.\n.\n.\n.\n.\n.\n인 척 ….],[],"[i_love_wooriwoori:이뿌다👏😍, go_ne_jin:@i_love_wo...",1
5,2021년 5월 6일,익선동,[간만에 🙈\n.\n.\n.\n.\n.\n.\n.\n.\n.\n머리끈 너무 거슬리는데],[],"[i_love_wooriwoori:😍 이뿌다, go_ne_jin:@i_love_wo...",1
6,2021년 3월 15일,Bar:eve - 바이브,[이 포즈 이제 컨셉✨],[],"[__j.euniii__:여긴 어댜 맛있어?, go_ne_jin:@_jungeun....",1
7,2020년 11월 11일,롯데월드 (lotteworld),[늙어서 많이 못놀게따🙈\n.\n.\n.\n.\n.\n.\n...,[],"[uoah.ns_:왜 나 안데려갔어 -3-, go_ne_jin:@tn_ns99 언니...",1
8,2020년 9월 2일,모이핀,[안 올수가 없는 카페야\n.\n.\n.\n.\n.\n.\n...,[],"[zzae_v:여기가 모이핀이군, go_ne_jin:@zzae_v 누가 더 인생샷인...",1
9,2020년 8월 12일,해운대 - Haeundae,[바다는 힐링🌙\n.\n.\n.\n.\n.\n.\n.\n근데 비와서 추어],[],"[__j.euniii__:여행갔는데 홀쭉해졌는데?! 좋겠따 여러모로😏😏, xiyeh...",1


In [94]:
comment_sentiment_list = ['감정분석결과']

In [95]:
for i in range(1,insta_df.shape[0]):
    if insta_df['comment'][i] != [] :
        comment_sentiment=list(lgbm_clf.predict(tfidf.transform(insta_df['comment'][i])))
    if insta_df['content'][i] != [] :
        comment_sentiment=list(lgbm_clf.predict(tfidf.transform(insta_df['content'][i])))
    if insta_df['hashtag'][i] != [] :
        comment_sentiment=list(lgbm_clf.predict(tfidf.transform(insta_df['hashtag'][i])))
    comment_sentiment_list.append(max(comment_sentiment,key = comment_sentiment.count))
comment_sentiment_list

['감정분석결과', 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1]

<p> 0 : 기쁨 행복</p>
<p> 1 : 분노 불안 당황 놀람</p>
<p> 2: 슬픔 상처 </p>

In [102]:
comment_sentiment_df = pd.DataFrame(comment_sentiment_list)
comment_sentiment_df.columns = ["label"]
comment_sentiment_df

,label
0,감정분석결과
1,1
2,1
3,1
4,1
5,1
6,1
7,1
8,0
9,1


In [107]:
insta_df

,date,place,content,hashtag,comment
0,날짜,장소,본문내용,해시태그,댓글내용
1,2022년 2월 24일,어마무시 티라미수,[2022❤️],[],"[zzae_v:다 좋은데 신발은…?, go_ne_jin:@zzae_v 벗어 던져버렸..."
2,2021년 8월 29일,낙산사,[개강 전 finale\n.\n.\n.\n.\n.\n.\n.\n.\n.\n.\n혜진...,[],"[suuout:👶, go_ne_jin:@sweeni_54 뻬이뻬 K대면에서 만나…,..."
3,2021년 7월 11일,하이서울유스호스텔 Hi Seoul Youth Hostel,[나도 기본캠📸플래쉬 감성 ?\n.\n.\n.\n.\n.\n.\n.\n.\n포둉포둉...,[],"[perarduaadastra__:어디가 포동포동이야 전혀 😮, go_ne_jin:..."
4,2021년 7월 11일,하이서울유스호스텔 Hi Seoul Youth Hostel,[글램핑\n.\n.\n.\n.\n.\n.\n.\n.\n.\n.\n.\n.\n인 척 ….],[],"[i_love_wooriwoori:이뿌다👏😍, go_ne_jin:@i_love_wo..."
5,2021년 5월 6일,익선동,[간만에 🙈\n.\n.\n.\n.\n.\n.\n.\n.\n.\n머리끈 너무 거슬리는데],[],"[i_love_wooriwoori:😍 이뿌다, go_ne_jin:@i_love_wo..."
6,2021년 3월 15일,Bar:eve - 바이브,[이 포즈 이제 컨셉✨],[],"[__j.euniii__:여긴 어댜 맛있어?, go_ne_jin:@_jungeun...."
7,2020년 11월 11일,롯데월드 (lotteworld),[늙어서 많이 못놀게따🙈\n.\n.\n.\n.\n.\n.\n...,[],"[uoah.ns_:왜 나 안데려갔어 -3-, go_ne_jin:@tn_ns99 언니..."
8,2020년 9월 2일,모이핀,[안 올수가 없는 카페야\n.\n.\n.\n.\n.\n.\n...,[],"[zzae_v:여기가 모이핀이군, go_ne_jin:@zzae_v 누가 더 인생샷인..."
9,2020년 8월 12일,해운대 - Haeundae,[바다는 힐링🌙\n.\n.\n.\n.\n.\n.\n.\n근데 비와서 추어],[],"[__j.euniii__:여행갔는데 홀쭉해졌는데?! 좋겠따 여러모로😏😏, xiyeh..."


In [109]:
pd.merge(insta_df,comment_sentiment_df,left_index= True, right_index=True )

,date,place,content,hashtag,comment,label
0,날짜,장소,본문내용,해시태그,댓글내용,감정분석결과
1,2022년 2월 24일,어마무시 티라미수,[2022❤️],[],"[zzae_v:다 좋은데 신발은…?, go_ne_jin:@zzae_v 벗어 던져버렸...",1
2,2021년 8월 29일,낙산사,[개강 전 finale\n.\n.\n.\n.\n.\n.\n.\n.\n.\n.\n혜진...,[],"[suuout:👶, go_ne_jin:@sweeni_54 뻬이뻬 K대면에서 만나…,...",1
3,2021년 7월 11일,하이서울유스호스텔 Hi Seoul Youth Hostel,[나도 기본캠📸플래쉬 감성 ?\n.\n.\n.\n.\n.\n.\n.\n.\n포둉포둉...,[],"[perarduaadastra__:어디가 포동포동이야 전혀 😮, go_ne_jin:...",1
4,2021년 7월 11일,하이서울유스호스텔 Hi Seoul Youth Hostel,[글램핑\n.\n.\n.\n.\n.\n.\n.\n.\n.\n.\n.\n.\n인 척 ….],[],"[i_love_wooriwoori:이뿌다👏😍, go_ne_jin:@i_love_wo...",1
5,2021년 5월 6일,익선동,[간만에 🙈\n.\n.\n.\n.\n.\n.\n.\n.\n.\n머리끈 너무 거슬리는데],[],"[i_love_wooriwoori:😍 이뿌다, go_ne_jin:@i_love_wo...",1
6,2021년 3월 15일,Bar:eve - 바이브,[이 포즈 이제 컨셉✨],[],"[__j.euniii__:여긴 어댜 맛있어?, go_ne_jin:@_jungeun....",1
7,2020년 11월 11일,롯데월드 (lotteworld),[늙어서 많이 못놀게따🙈\n.\n.\n.\n.\n.\n.\n...,[],"[uoah.ns_:왜 나 안데려갔어 -3-, go_ne_jin:@tn_ns99 언니...",1
8,2020년 9월 2일,모이핀,[안 올수가 없는 카페야\n.\n.\n.\n.\n.\n.\n...,[],"[zzae_v:여기가 모이핀이군, go_ne_jin:@zzae_v 누가 더 인생샷인...",0
9,2020년 8월 12일,해운대 - Haeundae,[바다는 힐링🌙\n.\n.\n.\n.\n.\n.\n.\n근데 비와서 추어],[],"[__j.euniii__:여행갔는데 홀쭉해졌는데?! 좋겠따 여러모로😏😏, xiyeh...",1
